In [14]:
# Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [15]:
# Import COVID-19 NYC Numbers
covid = "Data/COVID-19 by NY Metro Zip Code.csv"
covid_df = pd.read_csv(covid)
covid_df.head()

,ZIP_CODE,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
0,10001,Chelsea/NoMad/West Chelsea,Manhattan,421,1786.70,23563.03,25,106.10,7.05,5968
1,10002,Chinatown/Lower East Side,Manhattan,1233,1606.40,76755.41,160,208.45,10.11,12201
2,10003,East Village/Gramercy/Greenwich Village,Manhattan,516,959.08,53801.62,34,63.20,5.46,9451
3,10004,Financial District,Manhattan,39,1068.32,3650.61,1,27.39,6.07,642
4,10005,Financial District,Manhattan,79,940.91,8396.11,2,23.82,5.25,1504


In [16]:
# Import Zillow File
zillow = 'Data/Zillow by Zip Code (Metro & US).csv'
zillow_df = pd.read_csv(zillow)
zillow_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/1996,...,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020,5/31/2020,6/30/2020,Average 2020 YTD
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,233265.0,...,1234262,1229890,1226466,1208024,1182758,1150900,1134880,1120949,1112549,1151677
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,211748.0,...,493485,492514,491726,491562,492618,494017,494766,494546,494435,493657
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,245773.0,...,1153259,1156287,1175142,1193746,1205413,1203165,1209735,1211403,1212520,1205997
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,200430.0,...,336159,336142,336234,335959,336153,336611,337678,338602,339179,337364
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,286382.0,...,645348,643973,642628,642209,642227,642454,641440,640355,639311,641333


In [17]:
# Set index for covid dataframe
covid_df.set_index("ZIP_CODE", inplace = True)
covid_df.head()

,NEIGHBORHOOD_NAME,BOROUGH_GROUP,COVID_CASE_COUNT,COVID_CASE_RATE,POP_DENOMINATOR,COVID_DEATH_COUNT,COVID_DEATH_RATE,PERCENT_POSITIVE,TOTAL_COVID_TESTS
ZIP_CODE,,,,,,,,,
10001,Chelsea/NoMad/West Chelsea,Manhattan,421,1786.70,23563.03,25,106.10,7.05,5968
10002,Chinatown/Lower East Side,Manhattan,1233,1606.40,76755.41,160,208.45,10.11,12201
10003,East Village/Gramercy/Greenwich Village,Manhattan,516,959.08,53801.62,34,63.20,5.46,9451
10004,Financial District,Manhattan,39,1068.32,3650.61,1,27.39,6.07,642
10005,Financial District,Manhattan,79,940.91,8396.11,2,23.82,5.25,1504


In [19]:
# Transform Zillow dataframe
zillow_cols = ["RegionName", "1/31/2020", "2/29/2020", "3/31/2020", "4/30/2020", "5/31/2020", "6/30/2020", "Average 2020 YTD"]
zillow_transformed = zillow_df[zillow_cols].copy()

# Rename the column headers
zillow_transformed = zillow_transformed.rename(columns={"RegionName":"ZIP_CODE",
                                                       "1/31/2020": "JAN_2020",
                                                       "2/29/2020": "FEB_2020",
                                                       "3/31/2020": "MAR_2020",
                                                       "4/30/2020": "APR_2020",
                                                       "5/31/2020": "MAY_2020",
                                                       "6/30/2020": "JUN_2020",
                                                       "Average 2020 YTD": "AVERAGE_2020_YTD"})

# Set index
zillow_transformed.set_index("ZIP_CODE", inplace=True)
zillow_transformed.head()

,JAN_2020,FEB_2020,MAR_2020,APR_2020,MAY_2020,JUN_2020,AVERAGE_2020_YTD
ZIP_CODE,,,,,,,
10025,1208024,1182758,1150900,1134880,1120949,1112549,1151677
60657,491562,492618,494017,494766,494546,494435,493657
10023,1193746,1205413,1203165,1209735,1211403,1212520,1205997
77494,335959,336153,336611,337678,338602,339179,337364
60614,642209,642227,642454,641440,640355,639311,641333


In [20]:
# Create database connection 
connection_string = "postgres:postgres@localhost:5432/covid-19/zillow"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
# Confirm tables
engine.table_names()

['covid19', 'zillow']

In [23]:
# Load DataFrames into database
covid_df.to_sql(name='covid19', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "ZIP_CODE" of relation "covid19" does not exist
LINE 1: INSERT INTO covid19 ("ZIP_CODE", "NEIGHBORHOOD_NAME", "BOROU...
                             ^

[SQL: INSERT INTO covid19 ("ZIP_CODE", "NEIGHBORHOOD_NAME", "BOROUGH_GROUP", "COVID_CASE_COUNT", "COVID_CASE_RATE", "POP_DENOMINATOR", "COVID_DEATH_COUNT", "COVID_DEATH_RATE", "PERCENT_POSITIVE", "TOTAL_COVID_TESTS") VALUES (%(ZIP_CODE)s, %(NEIGHBORHOOD_NAME)s, %(BOROUGH_GROUP)s, %(COVID_CASE_COUNT)s, %(COVID_CASE_RATE)s, %(POP_DENOMINATOR)s, %(COVID_DEATH_COUNT)s, %(COVID_DEATH_RATE)s, %(PERCENT_POSITIVE)s, %(TOTAL_COVID_TESTS)s)]
[parameters: ({'ZIP_CODE': 10001, 'NEIGHBORHOOD_NAME': 'Chelsea/NoMad/West Chelsea', 'BOROUGH_GROUP': 'Manhattan', 'COVID_CASE_COUNT': 421, 'COVID_CASE_RATE': 1786.7, 'POP_DENOMINATOR': 23563.03, 'COVID_DEATH_COUNT': 25, 'COVID_DEATH_RATE': 106.1, 'PERCENT_POSITIVE': 7.05, 'TOTAL_COVID_TESTS': 5968}, {'ZIP_CODE': 10002, 'NEIGHBORHOOD_NAME': 'Chinatown

In [24]:
# Load DataFrames into database
zillow_transformed.to_sql(name='zillow', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "ZIP_CODE" of relation "zillow" does not exist
LINE 1: INSERT INTO zillow ("ZIP_CODE", "JAN_2020", "FEB_2020", "MAR...
                            ^

[SQL: INSERT INTO zillow ("ZIP_CODE", "JAN_2020", "FEB_2020", "MAR_2020", "APR_2020", "MAY_2020", "JUN_2020", "AVERAGE_2020_YTD") VALUES (%(ZIP_CODE)s, %(JAN_2020)s, %(FEB_2020)s, %(MAR_2020)s, %(APR_2020)s, %(MAY_2020)s, %(JUN_2020)s, %(AVERAGE_2020_YTD)s)]
[parameters: ({'ZIP_CODE': 10025, 'JAN_2020': 1208024, 'FEB_2020': 1182758, 'MAR_2020': 1150900, 'APR_2020': 1134880, 'MAY_2020': 1120949, 'JUN_2020': 1112549, 'AVERAGE_2020_YTD': 1151677}, {'ZIP_CODE': 60657, 'JAN_2020': 491562, 'FEB_2020': 492618, 'MAR_2020': 494017, 'APR_2020': 494766, 'MAY_2020': 494546, 'JUN_2020': 494435, 'AVERAGE_2020_YTD': 493657}, {'ZIP_CODE': 10023, 'JAN_2020': 1193746, 'FEB_2020': 1205413, 'MAR_2020': 1203165, 'APR_2020': 1209735, 'MAY_2020': 1211403, 'JUN_2020': 1212520, 'AVERAGE_2020_YTD': 1205997